In [20]:
#from google.colab import drive
#drive.mount('/content/drive')
# Imports
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [72]:
def save_model(epoch, model, optimizer, loss, name):
  torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
  }, name+'.pth')

def load_model(model, optimizer, epoch, loss, name):
  checkpoint = torch.load(name+'.pth')
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']
  return model, optimizer, epoch, loss

def loadData(trainPath, testPath, batch_size):

  transformer = transforms.Compose([
    transforms.Resize((256,256)),  #
    #transforms.CenterCrop(224),  #
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])

  train_loader = DataLoader(
      torchvision.datasets.ImageFolder(trainPath, transform = transformer),
      batch_size=batch_size,
      shuffle=True,
      drop_last=True)

  test_loader = DataLoader(
      torchvision.datasets.ImageFolder(testPath, transform = transformer),
      batch_size=batch_size,
      shuffle=False,
      drop_last=True)

  return train_loader, test_loader

In [64]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    #Canales de entrada, Canales de salida, Tamaño de Kernel, Stride y Padding
    self.conv1 = nn.Conv2d(3, 16, 3, 1, 0)
    self.conv2 = nn.Conv2d(16, 64, 3, 1, 0)
    self.conv3 = nn.Conv2d(64, 128, 3, 1, 0)
    self.dropout = nn.Dropout(0.2)
    self.fc1 = nn.Linear(128*4*4, 512)
    self.fc2 = nn.Linear(512, 256)
    
  def forward(self, x):
    #Entrada (batch, channels, ancho, largo)
    # (64, 3, 256, 256)
    x = torch.relu(self.conv1(x))
    x = torch.max_pool2d(x, 2, 2) #input, kernel_size, stride
    # (64, 6, 127, 127)
    x = torch.relu(self.conv2(x))
    x = torch.max_pool2d(x, 2, 2)
    # (64, 16, 62, 62)
    x = torch.relu(self.conv3(x))
    x = self.dropout(x)
    x = torch.max_pool2d(x, 2, 2)
    # (64, 64, 30, 30)
    x = x.view(-1, 128*4*4)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.softmax(x, dim=1)

    return x

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is Torch Cuda Available? " + str(torch.cuda.is_available()))
print("Device: " + str(device))

Is Torch Cuda Available? True
Device: cuda


In [73]:
trainPath = "Covid19-dataset/train"
testPath = "Covid19-dataset/test"
batch_size = 64
trainloader, testloader = loadData(trainPath, testPath, batch_size)

In [75]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Entrenamiento del modelo
num_epochs = 25
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs.size())
        print(labels.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

tensor([[[[-0.1922, -0.2941, -0.4275,  ..., -0.6784, -0.7098, -0.7020],
          [-0.2078, -0.3333, -0.4745,  ..., -0.6627, -0.6784, -0.6863],
          [-0.2471, -0.3804, -0.4824,  ..., -0.6314, -0.6549, -0.6627],
          ...,
          [ 0.8118,  0.8118,  0.8588,  ...,  0.4196,  0.4353,  0.3569],
          [ 0.8353,  0.8353,  0.8824,  ...,  0.4196,  0.4431,  0.4118],
          [ 0.8196,  0.8431,  0.8745,  ...,  0.3098,  0.3804,  0.3412]],

         [[-0.1922, -0.2941, -0.4275,  ..., -0.6784, -0.7098, -0.7020],
          [-0.2078, -0.3333, -0.4745,  ..., -0.6627, -0.6784, -0.6863],
          [-0.2471, -0.3804, -0.4824,  ..., -0.6314, -0.6549, -0.6627],
          ...,
          [ 0.8118,  0.8118,  0.8588,  ...,  0.4196,  0.4353,  0.3569],
          [ 0.8353,  0.8353,  0.8824,  ...,  0.4196,  0.4431,  0.4118],
          [ 0.8196,  0.8431,  0.8745,  ...,  0.3098,  0.3804,  0.3412]],

         [[-0.1922, -0.2941, -0.4275,  ..., -0.6784, -0.7098, -0.7020],
          [-0.2078, -0.3333, -

ValueError: Expected input batch_size (3600) to match target batch_size (64).